# Assignment 4: List Generation for Experiments
## Computational Methods in Psychology (and Neuroscience)
### Psychology 4500/7559 --- Fall 2020

# Objectives

Upon completion of this assignment, the student will have:

1. Read in a stimulus pool from a file.

2. Randomly generated lists to use in a experiment.

3. Written the lists out to files for future use.


# Assignment

* Write code in a Jupyter notebook (after making a copy and renaming it to have your userid in the title --- e.g., A04_ListGen_mst3k).

## Design

Your assignment is to write a script that reads in a pool of stimuli
and creates lists of dictionaries that you will later present to
participants as part of an experiment.  

The script should be configurable such that you can specify different
numbers of lists and trials, along with other details specific to the
experiment you decide to do.

Each dictionary represents a trial and should contain all the
information necessary to identify the stimulus to be presented,
details about that stimulus, and the condition in which to present it.
This information will be experiment-specific, as outlined below.

You have three options for your experiment.  Please select **one** of
the following experiments, keeping in mind that your next assignment
will be to code the experiment presentation and response collection
for the lists you generate from this assignment.

  
* ***When you are done, save this notebook as HTML (`File -> Download as -> HTML`) and upload it to the matching assignment on UVACollab.***  

## Option 1: Valence Study

The main question of this study is whether recognition memory for
words depends on the emotional or affective valence of those words.
Participants will study lists of positive (+), negative (-), and
neutral (~) words and then, after a short delay, they will be given a
recognition test over all the studied target words plus a matched set
of non-studied lures.  The stimuli are contained in three separate CSV
files:

- [Positive Pool](./pos_pool.csv)
- [Negative Pool](./neg_pool.csv)
- [Neutral Pool](./neu_pool.csv)

You will need to read these files in as lists of dictionaries (hint,
use the ``DictReader`` from the ``csv`` module that was covered in
class.)  Use these pools to create lists of trials for two
experimental conditions: pure or mixed.  In the *pure* condition,
all of the trials should be words from the same valence (be sure to
have the same number of positive, negative, and neutral pure lists.)
In the *mixed* condition, each list should contain an equal number of
positive, negative, and neutral words in *random* order (hint, use the
``shuffle`` function provided by the ``random`` module.) 

You will need to generate a matching test list for each study list
that includes all the studied items, plus a set of lures that match
the valence of the studied words.

Be sure to add in information to each trial dictionary that identifies
the word, its valence, the condition of the list, and whether it is a
target or a lure.  Feel free to add in more information if you would
like.


In [1]:
import csv
import random

pos_pool = csv.DictReader(open('pos_pool.csv', 'r'))
neg_pool = csv.DictReader(open('neg_pool.csv', 'r'))
neu_pool = csv.DictReader(open('neu_pool.csv', 'r'))
pos_list=[]
for l in pos_pool:
    x = {}
    x['stimulus'] = l['description']
    x['valence'] = 'POS'
    pos_list.append(x)
neg_list =[]
for l in neg_pool:
    x = {}
    x['stimulus'] = l['description']
    x['valence'] = 'NEG'
    neg_list.append(x)
neu_list=[]
for l in neu_pool:
    x = {}
    x['stimulus'] = l['description']
    x['valence'] = 'NEU'
    neu_list.append(x)
    
random.shuffle(pos_list)
random.shuffle(neg_list)
random.shuffle(neu_list)

def gen_pure(num_study, pool_list):
    pure_study = []
    pure_test = []
    pure = {} 
    for i in range(num_study):
        l_dict = pool_list[int(i + ((num_study/3)* 2))]
        x = {}
        x['stimulus'] = l_dict['stimulus']
        x['valence'] = l_dict['valence']
        x['cond'] = 'PURE'
        x['novelty'] = 'TARGET'
        pure_test.append(x)
        pure_study.append(x)
    for j in range(num_study):
        l_dict = pool_list[int(j + ((num_study/3)* 2) + num_study)]
        x = {}
        x['stimulus'] = l_dict['stimulus']
        x['valence'] = l_dict['valence']
        x['cond'] = 'PURE'
        x['novelty'] = 'LURE'
        pure_test.append(x)
    random.shuffle(pure_test)
    pure['study_list'] = pure_study
    pure['test_list'] = pure_test
    return pure

def gen_mixed(num_study):
    mix_study = []
    mix_test = []
    mix = {}
    for i in range(num_study):
        pos_dict = pos_list[i]
        x = {}
        x['stimulus'] = pos_dict['stimulus']
        x['valence'] = pos_dict['valence']
        x['cond'] = 'MIXED'
        x['novelty'] = 'TARGET'
        mix_test.append(x)
        mix_study.append(x)
        neg_dict = neg_list[i]
        y = {}
        y['stimulus'] = neg_dict['stimulus']
        y['valence'] = neg_dict['valence']
        y['cond'] = 'MIXED'
        y['novelty'] = 'TARGET'
        mix_test.append(y)
        mix_study.append(y)
        neu_dict = neu_list[i]
        z = {}
        z['stimulus'] = neu_dict['stimulus']
        z['valence'] = neu_dict['valence']
        z['cond'] = 'PURE'
        z['novelty'] = 'TARGET'
        mix_test.append(z)
        mix_study.append(z)
    for j in range(num_study):
        pos_dict = pos_list[j+num_study]
        x = {}
        x['stimulus'] = pos_dict['stimulus']
        x['valence'] = pos_dict['valence']
        x['cond'] = 'MIXED'
        x['novelty'] = 'LURE'
        mix_test.append(x)
        neg_dict = neg_list[j+num_study]
        y = {}
        y['stimulus'] = neg_dict['stimulus']
        y['valence'] = neg_dict['valence']
        y['cond'] = 'MIXED'
        y['novelty'] = 'LURE'
        mix_test.append(y)
        neu_dict = neu_list[j+num_study]
        z = {}
        z['stimulus'] = neu_dict['stimulus']
        z['valence'] = neu_dict['valence']
        z['cond'] = 'PURE'
        z['novelty'] = 'LURE'
    random.shuffle(mix_test)
    mix['study_list'] = mix_study
    mix['test_list'] = mix_test
    mix_test.append(z)
    return mix

def gen_blocks(num_blocks, num_study):
    blocks = []
    for i in range(num_blocks):
        mixed = gen_mixed(num_study)
        pos_pure = gen_pure(num_study*3,pos_list)
        neg_pure = gen_pure(num_study*3,neg_list)
        neu_pure = gen_pure(num_study*3,neu_list)
        random.shuffle(pos_list)
        random.shuffle(neg_list)
        random.shuffle(neu_list)
        blocks.append(pos_pure)
        blocks.append(neg_pure)
        blocks.append(neu_pure)
        blocks.append(mixed)
    return blocks


  




In [22]:
num_blocks = (int(input("How many blocks would you like for the study? ")))
num_study = (int(input("How many items would you like for EACH valance in the mixed study list? ")))
print("PLEASE NOTE: You will have " + str(num_study * 3) + " items for EACH study list and " + str(num_study * 6)  + " items for EACH test list")

# max number of items for num_study input is 26
# because the neutral pool has the least number of items which is 209 items
# the mixed test list would have 26 x 2 neutral test items (26 lures and 26 targets) items = 52
# using the same logic, the neutral pure test list would have ((26 x 3) x 2) items = 156
# 156 + 52 = 208... any more and we would go out of bounds 
assert (num_study < 27), "TOO MANY ITEMS. Please pick a smaller number of study items"

gen_blocks(num_blocks,num_study)


How many blocks would you like for the study? 2
How many items would you like for EACH valance in the mixed study list? 1
PLEASE NOTE: You will have 3 items for EACH study list and 6 items for EACH test list


[{'study_list': [{'stimulus': 'graduate',
    'valence': 'POS',
    'cond': 'PURE',
    'novelty': 'TARGET'},
   {'stimulus': 'river',
    'valence': 'POS',
    'cond': 'PURE',
    'novelty': 'TARGET'},
   {'stimulus': 'soothe',
    'valence': 'POS',
    'cond': 'PURE',
    'novelty': 'TARGET'}],
  'test_list': [{'stimulus': 'soothe',
    'valence': 'POS',
    'cond': 'PURE',
    'novelty': 'TARGET'},
   {'stimulus': 'people', 'valence': 'POS', 'cond': 'PURE', 'novelty': 'LURE'},
   {'stimulus': 'crown', 'valence': 'POS', 'cond': 'PURE', 'novelty': 'LURE'},
   {'stimulus': 'river',
    'valence': 'POS',
    'cond': 'PURE',
    'novelty': 'TARGET'},
   {'stimulus': 'diploma',
    'valence': 'POS',
    'cond': 'PURE',
    'novelty': 'LURE'},
   {'stimulus': 'graduate',
    'valence': 'POS',
    'cond': 'PURE',
    'novelty': 'TARGET'}]},
 {'study_list': [{'stimulus': 'dagger',
    'valence': 'NEG',
    'cond': 'PURE',
    'novelty': 'TARGET'},
   {'stimulus': 'unhappy',
    'valence': 'N

## Option 2: Scene Study

This study will test whether recognition memory for indoor and outdoor
scenes is modulated by the structure of the study lists.
Specifically, participants will study lists that either have indoor
and outdoor scenes that come in pure blocks or intermixed (similar to
the Valence study above).  The participants will then be given a
recognition test over all the studied target images plus a matched set
of non-studied lures.  You can access the lists of stimuli available:

- [Indoor Pool](./indoor.csv)
- [Outdoor Pool](./outdoor.csv)

You will need to read these files in as lists of dictionaries (hint,
use the ``DictReader`` from the ``csv`` module that was covered in
class.)  For the actual experiment we will give you the images that
are referenced by the file names in these pools, but for the list
generation you do not need the images, themselves and should identify
the image you will be presenting using the file name.  Use these pools
to create lists of trials for two experimental conditions: pure or
mixed.  In the *pure* condition, all of the trials should be images
from the same category (be sure to have the same number of indoor
and outdoor pure lists.)  In the *mixed* condition, each
list should contain an equal number of indoor and outdoor
images in *random* order (hint, use the ``shuffle`` function provided
by the ``random`` module.)

You will need to generate a matching test list for each study list
that includes all the studied items, plus a set of lures that match
the image categories from the studied items.

Be sure to add in information to each trial dictionary that identifies
the file name, the category of the image, the condition of the list,
and whether it is a target or a lure.


## Option 3: Your own study

You may also generate lists for a study specifically relevant to your
own work.  We are extremely supportive of this, but the study must be
approved by the professor.
